In [1]:
import time
import numpy as np
from math import pi
from scipy import optimize
import matplotlib.pyplot as plt

N, N_k= 40, 20 #Select the number of points given to calculated the approximated integral in each dimension
               #(N_k refers to number of Matsubara frequencies)

terminos=10 #modify

a_l=0.279894 #use the equilibrium_positions program to compute this parameter

def largo_alcance(k,terminos): #long-range interactions
    senos1, senos2 =0, 0
    for i in range(1,int(terminos+1/2)):
        senos1+=1/i**3*np.sin(k*i/2)**2
    
    if terminos>9:
        for i in range(int((terminos-2)/4)):
            senos2+=1/(2*i+1)**2
    else:
        for i in range(int((terminos-1)/2)):
            senos2+=1/(2*i+1)**2
        
    senos=senos2-senos1
    return ((1/a_l)**3*4*senos)**(1/2)

def integral_1r(m): #Tadpole integral
    
    def g_1(k,w):
        return ((1/a_l**3)*np.log(2))**(1/2)/(2*pi)**2*1/(w**2+largo_alcance(k,terminos)**2+m) 
    
    v1=np.repeat(np.linspace(-pi,pi,N_k),2*N)
    v2=np.tile(np.linspace(-2*pi*T*N,2*pi*T*N,2*N),N_k)
    return sum(g_1(v1,v2))*(2*pi*T)*(2*pi/N_k)

def integral_2r(m): #Sunrise integral
    
    def g_2(k1,k2,w1,w2):
        return ((1/a_l**3)*np.log(2))/(2*pi)**4*1/((w1**2+largo_alcance(k1,terminos)**2+m)*(w2**2+largo_alcance(k2,terminos)**2+m)*((w1+w2)**2+(largo_alcance(k1,terminos)+largo_alcance(k2,terminos))**2+m))
    
    b1=np.repeat(np.linspace(-pi,pi,N_k),N_k*(2*N)*(2*N))
    b2=np.tile(np.repeat(np.linspace(-pi,pi,N_k),(2*N)*(2*N)),N_k)
    b3=np.tile(np.repeat(np.linspace(-2*pi*T*N,2*pi*T*N,2*N),(2*N)),N_k*N_k)
    b4=np.tile(np.linspace(-2*pi*T*N,2*pi*T*N,2*N),(2*N)*N_k*N_k)
    
    return sum(g_2(b1,b2,b3,b4))*(2*pi*T)*(2*pi*T)*(2*pi/N_k)*(2*pi/N_k)

def integral_3r(m): #Coupling constant renormalisation
    def g_3(k,w):
        return ((1/a_l**3)*np.log(2))**(1/2)/(2*pi)**2*1/(w**2+largo_alcance(k,terminos)**2+m)**2
    v1=np.repeat(np.linspace(-pi,pi,N_k),2*N)
    v2=np.tile(np.linspace(-2*pi*T*N,2*pi*T*N,2*N),N_k)
    
    return sum(g_3(v1,v2))*(2*pi*T)*(2*pi/N_k)

#Self-consistenet equations to second order in loop expansion

def ecuacion_1r(m):
    return m-(lambd/2)*integral_1r(m)

def ecuacion_2r(m):
    return -mr2+m-(lambd**2/6)*integral_2r(m)

def lambda_bare(m):
    return lambd+(3*lambd**2/2)*integral_3r(m)

#m_r1=optimize.newton(ecuacion_2r,m0,tol=1e-2,full_output=True)

Critical line plots for different neighbour interactions values

In [2]:
lambdas=np.linspace(1,150,10)
terms=[5,15,30,60]
m_0C=np.empty([len(terms),len(lambdas)])
m_r1=np.empty([len(terms),len(lambdas)])
lambda_0=np.empty([len(terms),len(lambdas)])
m_r1[0,:]=[0,0.015,0.055,0.10,0.16,0.23,0.19,0.36,0.43,0.49]
indice_termino, indice_lambd=0, 0

T, mr2= 0.1, 0

for terminos in terms:
    print(terminos)
    for lambd in lambdas:
        m_r1[indice_termino,indice_lambd]=optimize.newton(ecuacion_2r,m_r1[indice_termino-1,indice_lambd],tol=1e-2)
        m_0C[indice_termino,indice_lambd]=ecuacion_1r(m_r1[indice_termino,indice_lambd])
        lambda_0[indice_termino,indice_lambd]=lambda_bare(m_r1[indice_termino,indice_lambd])
        print(m_0C[indice_termino,indice_lambd])
        indice_lambd+=1
    indice_termino+=1
    indice_lambd=0
    
valor=['j=5','j=15','j=30','j=60']

%matplotlib qt
for indice_plot in range(len(terms)):
    #plt.plot(m_0C[indice_plot,:],lambdas,'*-',label=valor[indice_plot])
    plt.plot(m_0C[indice_plot,:],lambda_0[indice_plot,:],'*-',label=valor[indice_plot])
plt.xlabel('$m_{0}^{2}$')
plt.ylabel('$\lambda_0$')
#plt.title('critical parameters depending of the number of neighbour interactions')
plt.legend()
plt.grid()
plt.show()

5


/Users/pablovinasmartinez/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: overflow encountered in multiply


-0.16236956010765533
-2.8234839017582742
-5.42852522112189
-7.971071920735847
-10.44626573748253
-12.850876681163138
-15.183230051021413
-17.44303881192807
-19.63114768152364
-21.74928290087371
15
-0.14721011367244688
-2.566866489001012
-4.951074421426535
-7.296561889292467
-9.600520169140538
-11.860647671613842
-14.075168488767815
-16.242827138311807
-18.362862985424478
-20.434969074080932
30
-0.13053640014577228
-2.2810775658129767
-4.410675044748411
-6.517914501812044
-8.601492692657818
-10.660228852466265
-12.693073322353628
-14.699113288618166
-16.67757563881029
-18.627827061889796
60
-0.12564647907134363
-2.196739993461382
-4.249975768853767
-6.284265709873496
-8.298593839869582
-10.29202345653584
-12.26370291912099
-14.212869957900418
-16.138854483414836
-18.0410799161471


Contour plots using the trapped-ion parameters

In [3]:
index_lambdas=10#100 select the number of points accoirding to the dessired precission
index_mr2=5#20
index_T=5#20

N, N_k= 20, 10

lambdas=np.linspace(0.01,15,index_lambdas)
MR2=np.linspace(0.0001,2,index_mr2) #MR2=[0]
t=np.linspace(0,0.3,index_T)


m_0C,lambda_0 =np.empty([index_lambdas,index_mr2,index_T]), np.empty([index_lambdas,index_mr2,index_T])

k, l, j= 0, 0, 0
iteraciones=0

startTime = time.time()
for T in t:
    for mr2 in MR2:
        m0=mr2
        for lambd in lambdas:
            print('Completed (%):',100*iteraciones/(index_lambdas*index_mr2*index_T),end='\r')
            m_r1=optimize.newton(ecuacion_2r,m0,tol=1e-2)
            m0=m_r1
            m_0C[l,k,j]=ecuacion_1r(m_r1)
            lambda_0[l,k,j]=lambda_bare(m_r1)
            iteraciones+=1
            l+=1
        k+=1
        l=0
    j+=1
    k=0

executionTime=(time.time()-startTime)
print('Execution time:' ,executionTime)

Execution time: 262.6928541660309


In [4]:
#trap frequencies and anisotropy parameter
w_x=2*pi*1.27e5
k_c=0.0024
k_min=1e-4

w_z = lambda k: (1/k)**(1/2)*w_x
w_z1 = w_z(k_c)
w_z2= w_z(k_min)

#fundamental parameters

h_bar=1.05457e-34
a_l=0.279894
k_B=1.38065e-23
e=1.60217e-19
eps_0=8.85419e-12
m_a=6.63e-26
zeta_3=1.202056

a=a_l*(e**2/(4*pi*eps_0*m_a*w_x**2))**(1/3)
lambda_ions=(h_bar/(m_a*w_x*a**2))*(1/(a_l)**3*1/np.log(2))**(1/2)

m02_ions = lambda w_z: (w_z/w_x)**2-1/(a_l)**3*(7/2)*zeta_3

#ranges of the dimensionless parameters for the contoutplot using trap parameters

m_02_ions1 = m02_ions(w_z1) 
m_02_ions2 = m02_ions(w_z2)

T_ions=k_B*1e-5/(h_bar*w_x)

#print(lambda_ions, T_ions, [m_02_ions1, m_02_ions2])

In [5]:
lambda_fijo = lambda_ions #Select a coupling constant from the valid range
m_0C_new, MR2_new, t_new= np.empty([index_mr2,index_T]), np.empty([index_mr2,index_T]), np.empty([index_mr2,index_T])

for i in range(index_mr2):
    for j in range(index_T):
        diferencias=abs(lambda_0[:,i,j]-lambda_fijo)
        index_lambda_fijo=np.where(diferencias==min(diferencias))
        m_0C_new[i,j]=m_0C[index_lambda_fijo,i,j]
    MR2_new[i,:]= 1/MR2[i] #MR[i]
for i in range(index_T):
    t_new[:,i]=t[i]
    
%matplotlib qt
plt.contourf(m_0C_new,t_new,MR2_new,100,cmap="plasma")
plt.xlabel('$m_0^{2}$')
plt.ylabel('$T$')
#plt.xlim(m_02_ions1, m_02_ions2) 
#plt.ylim((0, T_ions))
plt.grid()
cbar=plt.colorbar()
cbar.set_label('${m_{R2}}^2$')